In [2]:
# imports
import pandas as pd
import requests
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
# import data from previous part/defining variables
bikeDF = pd.read_csv('../data/Toronto_BikeCity.csv')

#creating variables for the request
FOURSQUARE_KEY = os.environ['FOURSQUARE_API_KEY']
url = "https://api.foursquare.com/v3/places/search"

#one list that will contain a bunch of dictionaries
FScontainer =[]

#loops through all the different bike stations, grabbing all the restaurants/bars within a 1000m radius
for index, row in bikeDF.iterrows():
    lat = row['Latitude']
    long = row['Longitude']

    # set parameters so it grabs specific information on bar/restuarants 
    params = {
        'll': f'{lat},{long}',
        'categoryId': '13003,13065',  # ID for dining and drinking
        "radius": '1000',
        'fields': 'name,location,categories,distance'
    }
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_KEY
    }

    #gather the data and turn it into a JSON
    request = requests.get(url, params=params, headers=headers)
    data=request.json()

    #create a more refined JSON which goes into the 'results' key (this will make it easier to call specific elements later)
    resultsJSON = data['results']
    #turn the refined JSON into a dataframe(easier to grab info from)
    resultsDF = pd.DataFrame(resultsJSON)

    #add a dictionary, containing all the relavent info, into the empty list
    #each entry in the list corresponds to a singular bike station
    FScontainer.append({
        'Latitude': lat,
        'Longitude': long,
        'Foursquare Name': resultsDF['name'],
        'Distance': resultsDF['distance']
        })



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [54]:
#requests and parsing of response had been completed in prior step/cell

,Latitude,Longitude,Foursquare Name,Distance
0,43.665269,-79.319796,0 Leslieville Farmers Market 1 ...,0 40 1 194 2 129 3 186 4 300 5...
1,43.671420,-79.445947,0 Balzac's Coffee Roasters 1 ...,0 284 1 339 2 270 3 581 4 505 5...
2,43.666224,-79.317693,0 Velotique 1 ...,0 17 1 100 2 223 3 180 4 257 5...
3,43.653236,-79.376716,0 Elgin & Winter Garden Theatre Ctr 1 ...,0 145 1 154 2 183 3 198 4 204 5...
4,43.663722,-79.380288,0 Black Eagle 1 ...,0 43 1 74 2 111 3 140 4 196 5...


Put your parsed results into a DataFrame

In [6]:
FOURSQUARE_DF = pd.DataFrame(FScontainer)
FOURSQUARE_DF.head()

,Latitude,Longitude,Foursquare Name,Distance
0,43.665269,-79.319796,0 Leslieville Farmers Market 1 ...,0 40 1 194 2 129 3 186 4 300 5...
1,43.671420,-79.445947,0 Balzac's Coffee Roasters 1 ...,0 284 1 339 2 270 3 581 4 505 5...
2,43.666224,-79.317693,0 Velotique 1 ...,0 17 1 100 2 223 3 180 4 257 5...
3,43.653236,-79.376716,0 Elgin & Winter Garden Theatre Ctr 1 ...,0 145 1 154 2 183 3 198 4 204 5...
4,43.663722,-79.380288,0 Black Eagle 1 ...,0 43 1 74 2 111 3 140 4 196 5...


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:

#creating variables for the request
YELP_KEY = os.environ['YELP_API_KEY']
url = "https://api.yelp.com/v3/businesses/search"

#one list that will contain a bunch of dictionaries
YELPcontainer =[]

#loops through all the different bike stations, grabbing all the restaurants/bars within a 1000m radius
for index, row in bikeDF.iterrows():
    lat = row['Latitude']
    long = row['Longitude']

    params = {
        'latitude': lat,
        'longitude': long,
        'radius': '1000',
        'categories': 'restaurants,bars',
    }
    headers = {
        'Authorization': 'Bearer '+ YELP_KEY
    }

    response = requests.get(url, params=params, headers=headers)


NameError: name 'bikeDF' is not defined

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating